# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании. Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Подготовка-признаков" data-toc-modified-id="Подготовка-признаков-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Подготовка признаков</a></span></li><li><span><a href="#Обучение-модели-линейной-регрессии" data-toc-modified-id="Обучение-модели-линейной-регрессии-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Обучение модели линейной регрессии</a></span></li></ul></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Алгоритм преобразования</a></span><ul class="toc-item"><li><span><a href="#Описание" data-toc-modified-id="Описание-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Описание</a></span></li><li><span><a href="#Обоснование" data-toc-modified-id="Обоснование-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Обоснование</a></span></li><li><span><a href="#Подбор-матрицы" data-toc-modified-id="Подбор-матрицы-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Подбор матрицы</a></span></li></ul></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

## Подготовка данных

### Загрузка данных

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_absolute_error

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_csv('datasets/insurance.csv')

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


В данных отсутствуют пропуски. Предобработка не требуется.

### Подготовка признаков

**Признаки:**
- пол
- возраст
- зарплата
- количество членов семьи

**Целевой признак:** количество страховых выплат клиенту за последние 5 лет.

In [5]:
features = data.drop(['Страховые выплаты'], axis=1)    
target = data['Страховые выплаты']    

Необходимо разделить доступные данные на обучающую и тестовую выборки.

In [6]:
features_valid, features_train, target_valid, target_train = \
    train_test_split(features, target, test_size=.75, random_state=1)

print('Обучающая выборка:', features_train.shape[0], 'строк')
print('Валидационная выборка:', features_valid.shape[0], 'строк')

Обучающая выборка: 3750 строк
Валидационная выборка: 1250 строк


### Обучение модели линейной регрессии

Несмотря на то, что количество возможных значений целевого признака невелико, перед нами стоит задача регрессии. В качестве  рабочей модели используется модель линейной регрессии. Однако предсказания модели могут иметь любое значение, в то время как количество страховых выплат является целым числом. Поэтому результаты расчетов должны быть округлены.

In [7]:
model = LinearRegression()
model.fit(features_train, target_train)

predictions = model.predict(features_valid)

Основной метрикой является $R^2$.

In [8]:
r2 = r2_score(target_valid, predictions.round())
print('R\u00b2 =', r2.round(2))

R² = 0.64


## Алгоритм преобразования

### Описание

Пробразование имеет вид

$X' = X \cdot P$

Матрица преобразования $P$ есть матрица линейного отображения. Мы не можем изменить количество признаков, поэтому $P$ является квадратной. Матрица преобразования должна быть линейно независимой, так как иначе один или несколько новых признаков окажутся линейной комбинацией других, что приведет к потере данных.

**Ответ:** Если $P$ обратимая, то результат предсказаний модели линейной регрессии будет неизменным.

### Обоснование

При доказательстве будут использованы следующие свойства матриц:

1. $A \cdot (BC) = (AB) \cdot C$
1. $(A B)^T = B^T A^T$
2. $(A B)^{-1} = B^{-1} A^{-1}$
3. $(A^T)^{-1} = (A^{-1})^T = A^{-T}$

После обучения предсказания вычисляются как

$a = \hat{X}\cdot w,$

где $\hat{X}$ - признаки, для которых необходимо сделать предсказания.

В результате обучения вектор весов линейной регрессии

$w = (X^T X)^{-1} X^T y,$

где $X$ - признаки из обучающей выборки,

 $\quad y$ - целевые признаки обучающей выборки.

Тогда можно опредлить предсказания по следующей формуле:

$a = \hat{X} \cdot(X^T X)^{-1} X^T y,$

Рассмотрим подробно процесс, происходящий при обучении с использованием преобразования.

**1. Преобразование признаков.**

$\quad \quad X' = X \cdot P$

$\quad \quad \hat{X'} = \hat{X} \cdot P$

где
\begin{equation}
 P = 
 \begin{pmatrix}
  p_{11} & \dots  & p_{1n}\\
  \vdots & \ddots & \vdots \\
  p_{n1} & \dots  & p_{nn}
 \end{pmatrix}
\end{equation}

**2. Обучение модели**

  Для обучения матрица признаков трансформируется.

\begin{equation*}
 \begin{pmatrix}
  x_{11} & \dots  & x_{1n}\\
  \vdots & \ddots & \vdots \\
  x_{n1} & \dots  & x_{nn}
 \end{pmatrix}
 \longrightarrow 
 \begin{pmatrix}
  1 & x_{11} & \dots  & x_{1n}\\
  1 & \vdots & \ddots & \vdots \\
  1 & x_{n1} & \dots  & x_{nn}
 \end{pmatrix}
\end{equation*}   

\begin{equation*}
 (w_1, w_2, \dots, w_n) \longrightarrow (w_0, w_1, w_2, \dots, w_n)
\end{equation*}   

Так как признаки были преобразованы матрицей $P$, необходимо трансформировать и ее:

\begin{equation*}
  \begin{pmatrix}
  p_{11} & \dots  & p_{1n}\\
  \vdots & \ddots & \vdots \\
  p_{n1} & \dots  & p_{nn}
 \end{pmatrix}
 \longrightarrow 
 \begin{pmatrix}
  1 & 0 & \dots  & 0\\
  0 & p_{11} & \dots  & p_{1n}\\
  0 & \vdots & \ddots & \vdots \\
  0 & p_{n1} & \dots  & p_{nn}
 \end{pmatrix},
\end{equation*}

тогда будет справедливо:

$\quad \quad X' = X \cdot P$

$\quad \quad \hat{X'} = \hat{X} \cdot P$

  В результате обучения определен вектор $w$:

$\quad \quad w = ((X P)^T (X P))^{-1} (X P)^T y = (P^T X^T X P)^{-1} P^T X^T y = (P^T \cdot (X^T X) \cdot P)^{-1} P^T X^T y = P^{-1} \cdot (X^T X)^{-1} \cdot P^{-T} P^T X^T y =$

$\quad \quad \quad= P^{-1} \cdot (X^T  X)^{-1} \cdot X^T y$

**3. Предсказания модели**

В результате расчетов 

$a = (\hat{X}P) \cdot w = (\hat{X} P) \cdot (P^{-1} (X^T X)^{-1} X^T y) = \hat{X} \cdot (P P^{-1}) \cdot (X^T X)^{-1} X^T y = \hat{X} (X^T X)^{-1} X^T y$

Данное выражение совпадает с тем, которое было получено без преобразования признаков. То есть преобразование не влияет на результат обучения линейной модели.

Важным для данного вывода свойством матрицы $P$ является то, что для нее существует обратная: то есть она является квадратной и невырожденной. Эти же свойства разумны с точки зрения постановки задачи. Таким образом достаточно обеспечить для матрицы $P$ выполнение этих условий.

### Подбор матрицы

Важным условием результирующих признаков является то, что по ним сложно восстановить данные. У каждого из признаков есть свои особенности, которые должны быть нивелированы: 

- `Пол` является категориальным признаком
- `Возраст` и `Зарплата` имеют большие амплитуды по сравнению с другими признаками
- `Члены семьи` имеют весьма ограниченный набор значений.

После преобразования признаки должны:
- Иметь сопоставимые значения
- Не быть дискретными

Простыми словами, в итоге матрица признаков должна иметь вид случайной матрицы.

Предлагается подобрать такие коэффициенты, чтобы, во-первых, масштабировать признаки, во-вторых, каждый новый признак имел вклад от всех начальных. Сформируем матрицу следующим образом:

1. Для каждого признака определим коэффициент масштабирования(1 / наибольшее значение).
2. Домножим строку из случайных чисел от 0 до 1 на этот коэффициент.

In [9]:
def get_transformation_matrix(features):

    rng = np.random.RandomState(1)
    n = len(features.columns)

    P = np.zeros((n, n))
    while np.linalg.det(P) == 0:
        i = 0
        for col in features.columns:
            factor = 1 / features[col].max()
            P[i] = factor * rng.rand(4)
            i += 1
    
    return P

P = get_transformation_matrix(features)
with np.printoptions(precision=4, suppress=True, floatmode='fixed'):
    print(P)

[[0.4170 0.7203 0.0001 0.3023]
 [0.0023 0.0014 0.0029 0.0053]
 [0.0000 0.0000 0.0000 0.0000]
 [0.0341 0.1464 0.0046 0.1117]]


## Проверка алгоритма

Для проверки необходимо обучить модель с измененными признаками:

In [10]:
model_upd = LinearRegression()
model_upd.fit(features_train.values.dot(P), target_train)

predictions_upd = model_upd.predict(features_valid.values.dot(P))

Сравним предсказания с начальными:

In [11]:
mean_absolute_error(predictions, predictions_upd).round(3)

0.0

Очевидно, что предсказания совпадают. Посчитаем метрику $R^2$ для новой модели:

In [12]:
r2_upd = r2_score(target_valid, predictions.round())
print('R\u00b2 =', r2.round(2))

R² = 0.64


In [13]:
r2 - r2_upd

0.0

Метрики совпадают.

**Вывод**

Как было показано ранее, для модели линейной регрессии использование матрицы преобразований не приводит к изменению предсказаний и, как следствие, к ухудшению качества модели.

## Выводы

В рамках проекта был предложен и обоснован метод преобразования данных для защиты персональной информации. Было доказано, что при трансформации данных при помощи домножения на обратимую матрицу результат предсказаний модели линейной регрессии не изменится. 

Так же была подобрана матрица преобразования так, чтобы сложно было восстановить начальные данные. Продемонстрировано, что предсказания модели не меняются, и, как следствие, качество модели линейной регрессии не ухудшается.